**Preprocessing e analisi dei dati**

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
from collections import Counter

import os

from IPython.display import Image
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from PIL import Image

import networkx as nx

from sklearn.preprocessing import MinMaxScaler
import random
import pickle

TEMPLATE = 'simple_white'

node_df = pd.read_csv("data/nodes.csv") # node & type
edge_df = pd.read_csv("data/edges.csv") # hero & comic
hero_net_df = pd.read_csv("data/hero-network.csv") # hero1 & hero2

Qualche info sulla rete per individuare gli eroi centrali, più popolari e collaborativi attraverso le misure di degree centrality e betweeneess centrality.

In [3]:
hero_net_df.info()

hn_info = nx.from_pandas_edgelist(hero_net_df, source = "hero1", target = "hero2")

# degree centrality 
DC = nx.degree_centrality(hn_info)
print("TOP 10 degree centrality")
counter = 0
for w in sorted(DC, key = DC.get , reverse = True):
    counter = counter + 1
    if counter == 11:
        break
    print(w,'{:0.2f}'.format(DC[w]))

print("\nTOP 10 betweenneess centrality")
# betweeness centrality
BC = nx.betweenness_centrality(hn_info)
counter = 0
for w in sorted(BC, key =BC.get,  reverse = True):
    counter = counter + 1
    if counter == 11:
        break
    print(w,"{:0.4f}".format(BC[w]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574467 entries, 0 to 574466
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hero1   574467 non-null  object
 1   hero2   574467 non-null  object
dtypes: object(2)
memory usage: 8.8+ MB
Graph with 6426 nodes and 167219 edges
CAPTAIN AMERICA 0.30
SPIDER-MAN/PETER PAR 0.27
IRON MAN/TONY STARK  0.24
THING/BENJAMIN J. GR 0.22
MR. FANTASTIC/REED R 0.21
WOLVERINE/LOGAN  0.21
HUMAN TORCH/JOHNNY S 0.21
SCARLET WITCH/WANDA  0.21
THOR/DR. DONALD BLAK 0.20
BEAST/HENRY &HANK& P 0.20
SPIDER-MAN/PETER PAR 0.0735
CAPTAIN AMERICA 0.0570
IRON MAN/TONY STARK  0.0372
WOLVERINE/LOGAN  0.0357
HAVOK/ALEX SUMMERS  0.0357
DR. STRANGE/STEPHEN  0.0292
THING/BENJAMIN J. GR 0.0254
HAWK 0.0248
HULK/DR. ROBERT BRUC 0.0239
MR. FANTASTIC/REED R 0.0238


Per una visualizzazione più pulita, sono stati estratti i 100 eroi con più connessioni. Il nodo della rete è l'eroe e l'edge rappresenta il numero di fumetti in cui due eroi appaiono insieme. Sui dati ricostruiti è stata creata una nuova rete dove:
- La grandezza del nodo rappresenta la centralità (basata sul page rank)
- Lo spessore dell'edge dipende dal numero di fumetti in cui i due eroi/nodi appaiono insieme
- Il nome dell'eroe con la centrality più alta è in grassetto

In [4]:
topn = 100
topn_hero = edge_df.groupby(['hero'])[['comic']].count().sort_values(by=['comic'], ascending=False).head(topn).index

h1_ = []; h2_ = []; cnt_ = [];
for comb in list(combinations(topn_hero, 2)):    
    temp1 = set(edge_df[edge_df['hero']==comb[0]]['comic'])
    temp2 = set(edge_df[edge_df['hero']==comb[1]]['comic'])
    cnt = len(temp1.intersection(temp2)) # apparizioni insieme dei due eroi    
    h1_.append(comb[0]); h2_.append(comb[1]); cnt_.append(cnt);
appto_df = pd.DataFrame({'H1':h1_, 'H2':h2_, 'CNT':cnt_})

display(appto_df.head())

,H1,H2,CNT
0,SPIDER-MAN/PETER PARKER,CAPTAIN AMERICA,145
1,SPIDER-MAN/PETER PARKER,IRON MAN/TONY STARK,95
2,SPIDER-MAN/PETER PARKER,THING/BENJAMIN J. GR,125
3,SPIDER-MAN/PETER PARKER,THOR/DR. DONALD BLAK,96
4,SPIDER-MAN/PETER PARKER,HUMAN TORCH/JOHNNY S,147


In [5]:
HERO_COLOR = {
    'CAPTAIN AMERICA':'darkblue',
    'IRON MAN':'gold',
    'SPIDER-MAN':'darkred',
    'HULK':'forestgreen',
    'THOR':'lightblue',
    'DR. STRANGE':'purple'
}

# creazione rete e inizializzazione grafo
marvel_net = nx.Graph() 
for i, row in appto_df.iterrows():
    marvel_net.add_edge(row['H1'], row['H2'], weight=row['CNT'])  # edge data

# posizioni nodi nella rete
pos_ = nx.spring_layout(marvel_net, seed=11)
cent_ = nx.pagerank(marvel_net, weight='weight') # page rank
cent_top = sorted(cent_.items(), key=lambda item: item[1], reverse=True)[:1] # page rank top 1

## funzione per creare un edge con un certo testo e spessore
def make_edge(x, y, text, width):
    return  go.Scatter(x=x, y=y, line=dict(width=width, color='lightgray'), hoverinfo='text', text=([text]), mode='lines')

# per ogni edge, viene creato un edge_trace e aggiunto alla lista
edge_trace = []
for edge in marvel_net.edges():    
    if marvel_net.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]
        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]
        trace  = make_edge([x0, x1, None], [y0, y1, None], None, width=5*(marvel_net.edges()[edge]['weight']/appto_df['CNT'].max()))
        edge_trace.append(trace)
                
# node trace
node_trace = go.Scatter(x=[], y=[], text=[], textposition="top center", textfont_size=10, mode='markers+text', hoverinfo='none',
                        marker=dict(color=[], size=[], line_width=[], line_color=[]))

# per ogni nodo nella rete, position e size vengono aggiunti a node_trace
for node in marvel_net.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    color = 'gray'
    line_width = 2
    line_color = 'darkgray'
    name_text = node
    
    if node in HERO_COLOR:
        color = HERO_COLOR[node]; line_color='black';
        
    if node in [v[0] for v in cent_top]:
        name_text = '<b>' + node + '</b>'
        
    node_trace['marker']['color'] += tuple([color])
    node_trace['marker']['size'] += tuple([int(400*cent_[node])]) # size nodo proporzionale al page rank
    node_trace['marker']['line_width'] += tuple([line_width])
    node_trace['marker']['line_color'] += tuple([line_color])
    node_trace['text'] += tuple([name_text])
    
    
# layout
layout = go.Layout(
    paper_bgcolor='rgba(255,255,255,1)', # transparent background
    plot_bgcolor='rgba(255,255,255,1)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)

# crea immagine grafo
fig = go.Figure(layout = layout)
# aggiunge gli edge traces
for trace in edge_trace:
    fig.add_trace(trace)
fig.add_trace(node_trace)
fig.update_layout(showlegend = False)
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)
fig.update_layout(title=f"<b>Top {topn} Marvel Heroes Network</b>")
fig.show()